In [1]:
import numpy as np
import pandas as pd

np.random.seed(42)   # reproducible

# Number of samples
n = 10

# Generate correlated 3D data
x1 = np.random.normal(0, 1, n)
x2 = 2 * x1 + np.random.normal(0, 0.5, n)
x3 = -x1 + 0.3 * x2 + np.random.normal(0, 0.2, n)

# Output column (linear combination + noise)
y = 3 * x1 - 2 * x2 + 0.5 * x3 + np.random.normal(0, 0.3, n)

# Create DataFrame
df = pd.DataFrame({
    "X1": x1,
    "X2": x2,
    "X3": x3,
    "Output_Y": y
})

print(df)


         X1        X2        X3  Output_Y
0  0.496714  0.761719  0.024931 -0.201343
1 -0.138264 -0.509393 -0.059709  1.129823
2  0.647689  1.416358 -0.209275 -0.998338
3  1.523030  2.089420 -1.181154 -0.517640
4 -0.234153 -1.330766 -0.273953  2.068858
5 -0.234137 -0.749418  0.031496  0.445919
6  1.579213  2.652010 -1.013809 -1.010627
7  0.767435  1.691993 -0.184697 -1.761932
8 -0.469474 -1.392961 -0.068542  0.944772
9  0.542560  0.378968 -0.487208  0.685198


In [2]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter3d(
    x=df["X1"],
    y=df["X2"],
    z=df["X3"],
    mode='markers',
    marker=dict(
        size=7,
        color=df["Output_Y"],
        colorscale='Viridis',
        colorbar=dict(title="Output_Y"),
        opacity=0.8
    )
))

fig.update_layout(
    title="3D Visualization of Input Features",
    scene=dict(
        xaxis_title="X1",
        yaxis_title="X2",
        zaxis_title="X3"
    )
)

fig.show()


# extract non target columns

In [3]:
X = df[["X1", "X2", "X3"]].values
X

array([[ 0.49671415,  0.76171946,  0.02493144],
       [-0.1382643 , -0.50939348, -0.059709  ],
       [ 0.64768854,  1.41635821, -0.20927543],
       [ 1.52302986,  2.08941959, -1.18115362],
       [-0.23415337, -1.33076567, -0.27395287],
       [-0.23413696, -0.74941768,  0.03149617],
       [ 1.57921282,  2.65201007, -1.01380851],
       [ 0.76743473,  1.69199312, -0.18469719],
       [-0.46947439, -1.39296081, -0.06854159],
       [ 0.54256004,  0.37896824, -0.48720832]])

## Standardize

In [4]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler(with_std=False)
X_meaned = scaler.fit_transform(X)


In [5]:
X_meaned

array([[ 0.04865304,  0.26092635,  0.36712333],
       [-0.58632541, -1.01018659,  0.28248289],
       [ 0.19962743,  0.91556511,  0.13291646],
       [ 1.07496874,  1.58862648, -0.83896172],
       [-0.68221449, -1.83155877,  0.06823902],
       [-0.68219807, -1.25021078,  0.37368806],
       [ 1.1311517 ,  2.15121696, -0.67161662],
       [ 0.31937362,  1.19120002,  0.1574947 ],
       [-0.9175355 , -1.89375392,  0.2736503 ],
       [ 0.09449893, -0.12182487, -0.14501643]])

## Covariance Matrix

In [6]:
cov_matrix = np.cov(X_meaned, rowvar=False)
print("Covariance Matrix:\n", cov_matrix)


Covariance Matrix:
 [[ 0.52274105  1.01530674 -0.25541782]
 [ 1.01530674  2.11289164 -0.41673212]
 [-0.25541782 -0.41673212  0.1835763 ]]


## Eigenvalues & Eigenvectors

In [7]:
eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)

print("Eigenvalues:\n", eigenvalues)
print("\nEigenvectors (columns):\n", eigenvectors)


Eigenvalues:
 [2.70141924 0.00335287 0.11443688]

Eigenvectors (columns):
 [[-0.43291881 -0.83010258 -0.35144133]
 [-0.8812312   0.30765193  0.3588619 ]
 [ 0.18977059 -0.46505913  0.8647006 ]]


## Sort by importance

In [8]:
idx = np.argsort(eigenvalues)[::-1]
eigenvalues = eigenvalues[idx]
eigenvectors = eigenvectors[:, idx]

print("Sorted Eigenvalues:\n", eigenvalues)
print("\nSorted Eigenvectors:\n", eigenvectors)


Sorted Eigenvalues:
 [2.70141924 0.11443688 0.00335287]

Sorted Eigenvectors:
 [[-0.43291881 -0.35144133 -0.83010258]
 [-0.8812312   0.3588619   0.30765193]
 [ 0.18977059  0.8647006  -0.46505913]]


## PCA Transformation

In [9]:
# Choose top 2 principal components
U = eigenvectors[:, :2]

# PCA projection
Z = X_meaned @ U   #  dot product

print("PCA Output (2D):\n", Z)


PCA Output (2D):
 [[-0.18133005  0.3939896 ]
 [ 1.19764618  0.08780463]
 [-0.86802337  0.37333705]
 [-2.02453167 -0.53314163]
 [ 1.92231998 -0.35851197]
 [ 1.46797613  0.11422787]
 [-2.51286944 -0.20629094]
 [-1.15809761  0.45142098]
 [ 2.11798419 -0.12051066]
 [ 0.03892566 -0.20232493]]


In [10]:
# Create PCA DataFrame
df_pca = pd.DataFrame(
    Z,
    columns=["PC1", "PC2"]
)

# Attach output column for visualization
df_pca["Output_Y"] = df["Output_Y"].values

print("PCA DataFrame:\n")
print(df_pca)


PCA DataFrame:

        PC1       PC2  Output_Y
0 -0.181330  0.393990 -0.201343
1  1.197646  0.087805  1.129823
2 -0.868023  0.373337 -0.998338
3 -2.024532 -0.533142 -0.517640
4  1.922320 -0.358512  2.068858
5  1.467976  0.114228  0.445919
6 -2.512869 -0.206291 -1.010627
7 -1.158098  0.451421 -1.761932
8  2.117984 -0.120511  0.944772
9  0.038926 -0.202325  0.685198


In [11]:
import plotly.express as px

fig = px.scatter(
    df_pca,
    x="PC1",
    y="PC2",
    color="Output_Y",
    title="2D PCA Projection",
    labels={
        "PC1": "Principal Component 1",
        "PC2": "Principal Component 2"
    },
    opacity=0.85
)

fig.update_layout(
    xaxis=dict(zeroline=True),
    yaxis=dict(zeroline=True)
)

fig.show()


# MODEL ACCURACY TEST BEFORE PCA

In [12]:
from sklearn.model_selection import train_test_split

X = df[["X1", "X2", "X3"]].values
y = df["Output_Y"].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)


In [13]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)


LinearRegression()

R² ≈ 1 → very good

R² ≈ 0 → useless

Negative R² → terrible

In [14]:
from sklearn.metrics import r2_score, mean_squared_error

y_pred = model.predict(X_test)

r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print("R² Score:", r2)
print("Mean Squared Error:", mse)


R² Score: -2.4645001943443536
Mean Squared Error: 0.28902464378804477


In [15]:
results = pd.DataFrame({
    "Actual_Y": y_test,
    "Predicted_Y": y_pred
})

print(results)


   Actual_Y  Predicted_Y
0  0.944772     1.614556
1  1.129823     0.684562
2  0.445919     0.915181


# Model AFTER PCA

APPLY PCA FROM SKLEARN

In [16]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

X_train_pca, X_test_pca, y_train, y_test = train_test_split(
    X_pca, y, test_size=0.3, random_state=42
)


In [17]:
model_pca = LinearRegression()
model_pca.fit(X_train_pca, y_train)

y_pred_pca = model_pca.predict(X_test_pca)

print("R² after PCA:", r2_score(y_test, y_pred_pca))
print("MSE after PCA:", mean_squared_error(y_test, y_pred_pca))


R² after PCA: -4.57971791310749
MSE after PCA: 0.4654858974770122


### Model became worse AFTER PCA .... why ?
The precise rule (THIS is the correct version)
❌ PCA tends to HURT when:

- Dataset is small

- Signal is already clean

- Task is supervised (regression / classification)

- Important features have low variance

- Relationship with y is not aligned with max variance

📌 PCA does not know y

✅ PCA tends to HELP when:

- Dataset is high-dimensional (many features)

- Features are noisy

- Strong multicollinearity

- p >> n (features ≫ samples)

- Task is unsupervised (clustering, visualization)

Or used as preprocessing, not final representation